<a href="https://colab.research.google.com/github/suhani121/Forest_fire/blob/main/1/Model/vit_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

!pip install -q split-folders tensorflow-addons

# MERGE FOLDERS
import os
import shutil
import splitfolders
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import MultiHeadAttention

#CHANGE THESE PATHS AS NEEDED
raw_base = "/content/drive/MyDrive/forest fire"   # Folders: fire, fire_mendel, non_fire, nofire_mendel
merged_data = "/content/drive/MyDrive/forest fire/merged"     # Final merged dataset folder

# Create target folders
os.makedirs(os.path.join(merged_data, 'fire'), exist_ok=True)
os.makedirs(os.path.join(merged_data, 'non_fire'), exist_ok=True)

# Move fire images
for folder in ['fire', 'fire_mendel']:
    folder_path = os.path.join(raw_base, folder)
    for img in os.listdir(folder_path):
        src = os.path.join(folder_path, img)
        dst = os.path.join(merged_data, 'fire', img)
        if os.path.isfile(src):
            shutil.copy(src, dst)

# Move non-fire images
for folder in ['non fire', 'nofire_mendel']:
    folder_path = os.path.join(raw_base, folder)
    for img in os.listdir(folder_path):
        src = os.path.join(folder_path, img)
        dst = os.path.join(merged_data, 'non_fire', img)
        if os.path.isfile(src):
            shutil.copy(src, dst)

print(" Merged fire and non-fire image folders.")

# SPLIT
split_output = "/content/drive/MyDrive/forest fire split"
splitfolders.ratio(merged_data, output=split_output, seed=1337, ratio=(0.8, 0.1, 0.1), move=False)

# IMAGE GENERATORS
IMG_SIZE = 224
BATCH_SIZE = 16

train_gen = ImageDataGenerator(rescale=1./255).flow_from_directory(
    f"{split_output}/train", target_size=(IMG_SIZE, IMG_SIZE), batch_size=BATCH_SIZE, class_mode='binary')

val_gen = ImageDataGenerator(rescale=1./255).flow_from_directory(
    f"{split_output}/val", target_size=(IMG_SIZE, IMG_SIZE), batch_size=BATCH_SIZE, class_mode='binary')

test_gen = ImageDataGenerator(rescale=1./255).flow_from_directory(
    f"{split_output}/test", target_size=(IMG_SIZE, IMG_SIZE), batch_size=BATCH_SIZE, class_mode='binary', shuffle=False)

#MODEL
def residual_block(x, filters, depthwise=False):
    shortcut = layers.Conv2D(filters, (1, 1), padding='same')(x)
    shortcut = layers.BatchNormalization()(shortcut)
    conv = layers.DepthwiseConv2D((3, 3), padding='same')(x) if depthwise else layers.Conv2D(filters, (3, 3), padding='same')(x)
    conv = layers.BatchNormalization()(conv)
    conv = layers.ReLU()(conv)
    conv = layers.DepthwiseConv2D((3, 3), padding='same')(conv) if depthwise else layers.Conv2D(filters, (3, 3), padding='same')(conv)
    conv = layers.BatchNormalization()(conv)
    out = layers.Add()([shortcut, conv])
    return layers.ReLU()(out)

def transformer_block(x, num_heads, ff_dim):
    attn_output = MultiHeadAttention(num_heads=num_heads, key_dim=ff_dim)(x, x)
    attn_output = layers.Dropout(0.1)(attn_output)
    out1 = layers.LayerNormalization(epsilon=1e-6)(x + attn_output)
    ffn = tf.keras.Sequential([layers.Dense(ff_dim, activation='relu'), layers.Dense(x.shape[-1])])
    ffn_output = ffn(out1)
    ffn_output = layers.Dropout(0.1)(ffn_output)
    return layers.LayerNormalization(epsilon=1e-6)(out1 + ffn_output)

class PatchExtractor(tf.keras.layers.Layer):
    def call(self, inputs):
        patches = tf.image.extract_patches(
            images=inputs,
            sizes=[1, 7, 7, 1],
            strides=[1, 7, 7, 1],
            rates=[1, 1, 1, 1],
            padding='VALID'
        )
        return patches

def vit_path(x):
    x = PatchExtractor()(x)
    x = layers.Reshape((-1, x.shape[-1]))(x)
    x = layers.Dense(64)(x)
    for _ in range(4):
        x = transformer_block(x, num_heads=4, ff_dim=64)
    x = layers.Flatten()(x)
    x = layers.Dense(64, activation='relu')(x)
    x = layers.Dense(32, activation='relu')(x)
    x = layers.Dense(16, activation='relu')(x)
    return x

def cnn_path(x):
    x = residual_block(x, 64)
    x = residual_block(x, 64, depthwise=True)
    x = residual_block(x, 64)
    x = residual_block(x, 64, depthwise=True)
    x = layers.GlobalMaxPooling2D()(x)
    x = layers.Dense(64, activation='relu')(x)
    x = layers.Dense(32, activation='relu')(x)
    x = layers.Dense(16, activation='relu')(x)
    return x

def build_model():
    inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    base_model = MobileNetV2(include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3), weights='imagenet')
    backbone = models.Model(inputs=base_model.input, outputs=base_model.get_layer('block_6_expand_relu').output)
    backbone_output = backbone(inputs)

    vit_features = vit_path(backbone_output)
    cnn_features = cnn_path(backbone_output)
    combined = layers.Add()([vit_features, cnn_features])
    combined = layers.Dense(16, activation='relu')(combined)
    combined = layers.Dense(8, activation='relu')(combined)
    outputs = layers.Dense(1, activation='sigmoid')(combined)

    return models.Model(inputs, outputs)

#TRAIN
model = build_model()
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()
history = model.fit(train_gen, validation_data=val_gen, epochs=20)

#TEST
loss, acc = model.evaluate(test_gen)
print(f" Test Accuracy: {acc:.4f}")


 Merged fire and non-fire image folders.



Copying files: 0 files [00:00, ? files/s]
Copying files: 1 files [00:00,  2.25 files/s]
Copying files: 11 files [00:00, 25.63 files/s]
Copying files: 21 files [00:00, 43.87 files/s]
Copying files: 29 files [00:16,  1.24 files/s]
Copying files: 38 files [00:16,  2.00 files/s]
Copying files: 47 files [00:16,  3.08 files/s]
Copying files: 57 files [00:17,  4.72 files/s]
Copying files: 67 files [00:17,  6.97 files/s]
Copying files: 77 files [00:17,  9.97 files/s]
Copying files: 87 files [00:17, 13.92 files/s]
Copying files: 98 files [00:17, 19.44 files/s]
Copying files: 108 files [00:17, 25.42 files/s]
Copying files: 118 files [00:17, 32.15 files/s]
Copying files: 128 files [00:17, 39.90 files/s]
Copying files: 137 files [00:17, 47.06 files/s]
Copying files: 147 files [00:18, 54.78 files/s]
Copying files: 157 files [00:18, 62.56 files/s]
Copying files: 166 files [00:18, 65.82 files/s]
Copying files: 175 files [00:18, 69.81 files/s]
Copying files: 185 files [00:18, 75.75 files/s]
Copying f

Found 4137 images belonging to 2 classes.
Found 584 images belonging to 2 classes.
Found 588 images belonging to 2 classes.


Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_2        │ (None, 28, 28,    │     65,920 │ input_layer_4[0]… │
│ (Functional)        │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ patch_extractor     │ (None, 4, 4,      │          0 │ functional_2[0][… │
│ (PatchExtractor)    │ 9408)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape (Reshape)   │ (None, 16, 9408)  │          0 │ patch_extractor[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 16, 64)    │    602,176 │ reshape[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 16, 64)    │     66,368 │ dense[0][0],      │
│ (MultiHeadAttentio… │                   │            │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 16, 64)    │          0 │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 16, 64)    │          0 │ dense[0][0],      │
│                     │                   │            │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalization │ (None, 16, 64)    │        128 │ add[0][0]         │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential          │ (None, 16, 64)    │      8,320 │ layer_normalizat… │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 28, 28,    │    110,656 │ functional_2[0][… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 16, 64)    │          0 │ sequential[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 28, 28,    │        256 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 16, 64)    │          0 │ layer_normalizat… │
│                     │                   │            │ dropout_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu (ReLU)        │ (None, 28, 28,    │          0 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 16, 64)    │        128 │ add_1[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 28, 28,    │     12,352 │ functional_2[0][… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 28, 28,    │     36,928 │ re_lu[0][0]       │
│                     │ 64)               │            │                 

...

 Non-trainable params: 5,440 (21.25 KB)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20


Copying files: 0 files [15:03, ? files/s]


259/259 ━━━━━━━━━━━━━━━━━━━━ 119s 218ms/step - accuracy: 0.8070 - loss: 0.4295 - val_accuracy: 0.8853 - val_loss: 0.2737
Epoch 2/20
259/259 ━━━━━━━━━━━━━━━━━━━━ 24s 92ms/step - accuracy: 0.9713 - loss: 0.0853 - val_accuracy: 0.9932 - val_loss: 0.0520
Epoch 3/20
259/259 ━━━━━━━━━━━━━━━━━━━━ 24s 92ms/step - accuracy: 0.9937 - loss: 0.0288 - val_accuracy: 0.9932 - val_loss: 0.0323
Epoch 4/20
259/259 ━━━━━━━━━━━━━━━━━━━━ 25s 96ms/step - accuracy: 0.9955 - loss: 0.0168 - val_accuracy: 0.9932 - val_loss: 0.0356
Epoch 5/20
259/259 ━━━━━━━━━━━━━━━━━━━━ 28s 106ms/step - accuracy: 0.9987 - loss: 0.0108 - val_accuracy: 0.9949 - val_loss: 0.0252
Epoch 6/20
259/259 ━━━━━━━━━━━━━━━━━━━━ 25s 95ms/step - accuracy: 0.9986 - loss: 0.0075 - val_accuracy: 0.9966 - val_loss: 0.0151
Epoch 7/20
259/259 ━━━━━━━━━━━━━━━━━━━━ 25s 95ms/step - accuracy: 0.9998 - loss: 0.0050 - val_accuracy: 0.9966 - val_loss: 0.0185
Epoch 8/20
259/259 ━━━━━━━━━━━━━━━━━━━━ 25s 97ms/step - accuracy: 0.9990 - loss: 0.0035 - val_accu